# Let’s embrace WebAssembly!

EuroPython 2018 - Edinburgh

Almar Klein

<img src='images/wasm.png' width=100 align=right>
<img src='images/python.png' width=100 align=right>


# What is WebAssembly?

<img src='images/wasm.png' align=right width=300>

(BTW: WebAssembly == WASM)

# WASM is an OPEN standard ...

<br>
<div style='display:flex; width:100%; justify-content:space-around; align-items:center;'>
<img src='images/group.png' style='height:200px; width:200px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

Collaborative effort by Mozilla, Google, Apple, Microsoft ...


# ... for executable code

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:200px; width:200px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

It's fast!


# It has a compact binary format

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:200px; width:200px;'>
<img src='images/check-square.png' style='height:100px; width:100px;'>
</div>
<br>

And a human readable counterpart:

```wasm
(module
(type $print (func (param i32))
(func $main
    (i32.const 42)
    (call $print)
)
(start $main)
)
```


# It's safe

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:100px; width:100px;'>
<img src='images/rocket.png' style='height:100px; width:100px;'>
<img src='images/file.png' style='height:100px; width:100px;'>
<img src='images/check-square.png' style='height:200px; width:200px;'>
</div>
<br>

Because browsers.

# WebAssembly is coming and it's awesome!

<br>
<div style='display:flex; width:100%'>
<img src='images/group.png' style='height:50px; width:50px;'>
<img src='images/rocket.png' style='height:50px; width:50px;'>
<img src='images/file.png' style='height:50px; width:50px;'>
<img src='images/check-square.png' style='height:50px; width:50px;'>
</div>
<br><br>

<center>
<img src='images/so-much-awesome-meme.jpg' width=400px>
</center>


# WebAssembly adoption

# Lua community

Let's write web apps in Lua!

<center><img src='images/celebrate.jpg' width=300></center>

# Rust community

Let's write apps in Rust!!

<center><img src='images/celebrate.jpg' width=300></center>



# JavaScript community


Yes, this may end our suffering!

<center><img src='images/celebrate.jpg' width=300></center>

# C++ community

Let's write web apps in C++!


<center><img src='images/celebrate.jpg' width=200px><img src='images/facepalm.jpg' width=200px>
</center>




# Python community

<br><br><br>

...


# WASM may not be obvious for Python

* Because Python is an *interpreted* language

# Three use-cases how we can embrace WASM

# Use case 1: Compile a Python interpreter



# Examples


* Pyodide: compiles CPython + numpy/pandas/matplotlib, to run in the browser
* PyPyJS
* MicroPython
* RsPython: Python interpreter written in Rust

Note: Python code is still run in a VM!

# Use case 2: Compile a subset of Python to WASM

# Use case 3: Run WASM in Python
    
    ... and allow that code to call into Python functions ...

# Rocket game

<center>
    <!-- <a href='https://thread-safe.nl/rocket/' target='new'> -->
    <a href='rocket.html' target='new'>
    <img src='images/github_rocket_wasm.png' width=900>
    </a>
</center>

# Single binary WASM file (58 KB)

<center>    
    <img src='images/github_rocket_wasm_html.png' width=600>
</center>



In [ ]:
from ppci import wasm

m = wasm.Module(open(r'rocket.wasm', 'rb'))
m

In [ ]:
m.show_interface()

In [ ]:
%display rocket.py

In [ ]:
import time
import math

class BaseRocketGame:
    
    def _instantiate(self):
        """ Instantiate the wasm module, using the objects' methods as imports.
        """
        env = {}
        for name in dir(self):
            if name.startswith('wasm_'):
                env[name[5:]] = getattr(self, name)
        imports = dict(env=env)
        
        import ppci.wasm.instantiate # TODO: UGLY
        self.wam = wasm.instantiate.instantiate(m, imports, target='python')
    
    def __init__(self):
        self._instantiate()
    
    def run(self, t=None):
        """ Enter the game's main loop.
        """
        self.wam.exports.resize(100, 100)
        et = time.time() + (1e12 if t is None else t)
        while time.time() < et:
            time.sleep(0.5)
            self.wam.exports.update(0.1)
            self.wam.exports.draw()
            
            # We never call these ...
            # self.wam.exports.toggle_shoot(b)
            # self.wam.exports.toggle_turn_left(b)
            # self.wam.exports.toggle_turn_right(b)
            # self.wam.exports.toggle_boost(b)
    
    def wasm_sin(self, a:float) -> float:
        return math.sin(a)
    
    def wasm_cos(self, a:float) -> float:
        return math.cos(a)
    
    def wasm_Math_atan(self, a:float) -> float:
        return math.atan(a)
    
    def wasm_clear_screen(self) -> None:
        print('clearing screen')
    
    def wasm_draw_bullet(self, x:float, y:float) -> None:
        print(f'There is a bullet at {x}, {y}')
    
    def wasm_draw_enemy(self, x:float, y:float) -> None:
        print(f'There is an enemy at {x}, {y}')
    
    def wasm_draw_particle(self, x:float, y:float, a:float) -> None:
        print(f'There is a particle at {x}, {y} angle {a}')
    
    def wasm_draw_player(self, x:float, y:float, a:float) -> None:
        print(f'The player is at {x}, {y} angle {a}')
    
    def wasm_draw_score(self, score:float) -> None:
        print(f'The score is {score}!')

# Run Rocket in Python

In [ ]:
game = BaseRocketGame()

In [ ]:
game.run(5)

# Run Rocket in Python with Qt

In [ ]:
from rocket_qt import QtRocketGame
import logging
logging.getLogger().setLevel(logging.WARN)
game = QtRocketGame()

In [ ]:
game.run()

# Run Rocket in Python with prompt_toolkit

Over SSH :)

# Is Rocket hard to play?

Let's make an AI!

In [1]:
from ppci import wasm

ai = wasm.Module(open('ai.wasm', 'rb'))

In [2]:
ai.show_interface()

Imports:
  env.atan:          [f32] -> [f32]
  env.cos:           [f32] -> [f32]
  env.sin:           [f32] -> [f32]
  env.toggle_shoot:  [i32] -> []
  env.toggle_turn_left: [i32] -> []
  env.toggle_turn_right: [i32] -> []
Exports:
  memory:            "memory"
  main:              [] -> [i32]
  clear_screen:      [] -> []
  draw_player:       [f32, f32, f32] -> []
  draw_enemy:        [f32, f32] -> []
